In [15]:
import foolbox as fb
import pandas as pd
import eagerpy as ep
import numpy as np

In [2]:
from tensorflow.keras.models import load_model

In [3]:
# White-box model
model = load_model('../victim/model.h5')

In [4]:
# Wrap to fb model
attacker_model = fb.models.TensorFlowModel(model, bounds=(0, 1))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [28]:
X_src = pd.read_csv('../victim/X.csv').values
y_src = pd.read_csv('../victim/y.csv').values

In [29]:
df = pd.DataFrame(X_src)
df['y'] = y_src

df = df[df.y == 1]

In [30]:
X_src = df.drop('y', axis=1).values
y_src = df.y.values

In [31]:
X_src = ep.from_numpy(attacker_model.dummy, X_src)
y_src = ep.from_numpy(attacker_model.dummy, y_src)

In [37]:
# Target
target = 0

criterion = fb.criteria.TargetedMisclassification(
    ep.from_numpy(attacker_model.dummy, np.array([target] * len(df)))
)

In [54]:
attack = fb.attacks.LinfPGD()
raw_adversarial, clipped_adversarial, success = attack(
    attacker_model,
    X_src,
    criterion=criterion,
    epsilons=0.0000000001
)

In [55]:
success

TensorFlowTensor(<tf.Tensor: shape=(19084,), dtype=bool, numpy=array([ True,  True,  True, ...,  True,  True,  True])>)